

# PC External

## Program for multiprocessing to print number of cores and specification of hardware devices

In [1]:
import multiprocessing as mp
print("number of processes: ", mp.cpu_count())

number of processes:  2


In [2]:
import platform

my_system = platform.uname()

print(f"System: {my_system.system}")
print(f"Node Name: {my_system.node}")
print(f"Release: {my_system.release}")
print(f"Version: {my_system.version}")
print(f"Machine: {my_system.machine}")
print(f"Processor: {my_system.processor}")

System: Linux
Node Name: a98c3b3538e3
Release: 6.1.85+
Version: #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
Machine: x86_64
Processor: x86_64


## Listing local devices

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15957911863931742156
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14626652160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9921728421011698350
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


## GPU utilization


In [6]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=7057ba860f6dc4023c54ba8dd9b91fc485c8339d25dd4cb216bff86c4249de63
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [7]:
import GPUtil
GPUtil.showUtilization()
GPUs = GPUtil.getGPUs()


| ID | GPU | MEM |
------------------
|  0 |  0% |  1% |


In [8]:
!cat /proc/meminfo

MemTotal:       13290460 kB
MemFree:         6355212 kB
MemAvailable:   11542172 kB
Buffers:          430484 kB
Cached:          4907296 kB
SwapCached:            0 kB
Active:           777916 kB
Inactive:        5631744 kB
Active(anon):       2400 kB
Inactive(anon):  1083772 kB
Active(file):     775516 kB
Inactive(file):  4547972 kB
Unevictable:           8 kB
Mlocked:               8 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               256 kB
Writeback:             0 kB
AnonPages:       1071900 kB
Mapped:           808488 kB
Shmem:             14284 kB
KReclaimable:     188876 kB
Slab:             249656 kB
SReclaimable:     188876 kB
SUnreclaim:        60780 kB
KernelStack:        6496 kB
PageTables:        16112 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645228 kB
Committed_AS:    3787488 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       84676 kB
VmallocChunk:    

In [10]:
import time as t
start = t.time()
print("time: ", (t.time()-start))

time:  5.269050598144531e-05


## Sequential Execution

In [12]:
import time as t
from timeit import default_timer as timer
import multiprocessing as mp
import numpy as np

x = [1,2,3,4]
start = timer()
y=0

for i in range(len(x)):
  y += x[i]

print(y)
print("time: ", (timer()-start))

10
time:  0.0015995820001535321


## Execution by chunks

In [15]:
from timeit import default_timer as timer
import multiprocessing as mp
import numpy as np

x = [1,2,3,4]
start = timer()
y=0

chunk_1 = x[:2]
chunk_2 = x[2:]

sum_1 = sum(chunk_1)
sum_2 = sum(chunk_2)
result = sum_1 + sum_2

print(result)
end1 = timer()-start
print("time: ",(end1))

10
time:  0.0018380809999598569


## checking process id

In [21]:
import multiprocessing as mp
# import os

def square(n):
    # print("Worker process id for {0}: {1}".format(n, os.getpid()))
    return (n*n)

if __name__ == "__main__":
    # input list
    mylist = [1,2,3,4,5]

    # creating a pool object
    p = mp.Pool()

    # map list to target function
    result = p.map(square, mylist)

    print(result)

[1, 4, 9, 16, 25]


## 4. square of the list


In [24]:
import multiprocessing as mp
from timeit import default_timer as timer

def square(n):
  return (n*n)

start=timer()

mylist = [1,2,3,4,5]

p = mp.Pool()

result = p.map(square, mylist)
end=timer()-start

print(result)
print(end)

[1, 4, 9, 16, 25]
0.031988102000013896


## 5. Factorial

In [29]:
import multiprocessing as mp
from timeit import default_timer as timer
from multiprocessing import pool

def factorial(n):
  result = 1
  for i in range(2, n):
      result *= i
  return result

def factorial_for_pool(start, end):
  result = 1
  for i in range(start, end + 1):
      result *= i
  return result

def factorial_process(n):
  chunk_size = n//mp.cpu_count()
  ranges = [(i * chunk_size + 1, min((i+1)*chunk_size,n)) for i in range(mp.cpu_count())]
  with mp.Pool(processes = mp.cpu_count()) as pool:
    results = pool.starmap(factorial_for_pool, ranges)
  final_result = 1
  for result in results:
    final_result *= result
  return final_result

def factorial_manual(n):
  def factorial_worker(start,end, result_list,idx):
    result = 1
    for i in range(start, end + 1):
      result *= i
    result_list[idx] = result

  chunk_size = n//mp.cpu_count()
  ranges = [(i * chunk_size + 1, min((i+1)*chunk_size,n)) for i in range(mp.cpu_count())]

  manager = mp.Manager()
  results = manager.list([1] * mp.cpu_count())
  processes = []

  for idx, (start,end) in enumerate(ranges):
    p = mp.Process(target=factorial_worker, args=(start,end,results,idx))
    processes.append(p)
    p.start()

  for p in processes:
    p.join()

  final_result = 1
  for result in results:
    final_result *= result
  return final_result

if __name__ == "__main__":
  numbers = [35000,40000,45000,50000,60000]
  seq_times = []
  pool_times = []
  manual_process_times = []

  for n in numbers:
    start = timer()
    factorial(n)
    seq_times.append(timer()-start)

    start = timer()
    factorial_process(n)
    pool_times.append(timer()-start)

    start = timer()
    factorial_manual(n)
    manual_process_times.append(timer()-start)

    print(f"Data size: {n}")
    print(f"Sequential Time: {seq_times[-1]}")
    print(f"Pool Time: {pool_times[-1]}")
    print(f"Process Time: {manual_process_times[-1]}")

Data size: 35000
Sequential Time: 0.23140311800034397
Pool Time: 0.17980205699950602
Process Time: 0.20366174799983128
Data size: 40000
Sequential Time: 0.313718936999976
Pool Time: 0.20905981899977633
Process Time: 0.24338471799910621
Data size: 45000
Sequential Time: 0.408906546000253
Pool Time: 0.2563938329994926
Process Time: 0.28615830300077505
Data size: 50000
Sequential Time: 0.503106850000222
Pool Time: 0.30808429699936823
Process Time: 0.34028640299948165
Data size: 60000
Sequential Time: 0.7302231289995689
Pool Time: 0.4367379309996977
Process Time: 0.46098663000066153


## 6. Even/odd

In [4]:
import multiprocessing as mp
# from timeit import default_timer as timer
import time
from multiprocessing import pool

def check_even_odd(n):
  return "Even" if n % 2 == 0 else "Odd"

def check_sequential(numbers):
  start = time.time()
  results = [check_even_odd(num) for num in numbers]
  return results

def check_pool(numbers):
  with mp.Pool() as pool:
    results = pool.map(check_even_odd, numbers)
  return results

def check_process(numbers):
  num_processes = mp.cpu_count()
  chunk_size = len(numbers) // num_processes + (len(numbers)% num_processes > 0)
  chunks  = [numbers[i*chunk_size: (i+1) * chunk_size] for i in range (num_processes)]

  def worker(numbers_chunk, results, idx):
    results[idx] = [check_even_odd(num) for num in numbers_chunk]

  with mp.Manager() as manager:
    results = manager.list([None] * num_processes)
    processes = []

    for idx, numbers_chunk in enumerate(chunks):
      p = mp.Process(target=worker, args=(numbers_chunk, results, idx))
      processes.append(p)
      p.start()

    for p in processes:
      p.join()

    return [result for chunk in results for result in chunk]

if __name__ == "__main__":
  numbers = [1,2,3,4,5,6]

  seq_times = []
  pool_times = []
  process_times = []

  for num in numbers:
    n = list(range(num))

    start = time.time()
    check_sequential(n)
    seq_times.append(time.time()-start)

    start = time.time()
    check_pool(n)
    pool_times.append(time.time()-start)

    start = time.time()
    check_process(n)
    process_times.append(time.time()-start)

    print(f"Data Size: {n}")
    print(f"Sequential Time: {seq_times[-1]:.4f} seconds")
    print(f"Pool Time: {pool_times[-1]:.4f} seconds")
    print(f"Process Time: {process_times[-1]:.4f} seconds")


Data Size: [0]
Sequential Time: 0.0000 seconds
Pool Time: 0.0289 seconds
Process Time: 0.0446 seconds
Data Size: [0, 1]
Sequential Time: 0.0000 seconds
Pool Time: 0.0308 seconds
Process Time: 0.0440 seconds
Data Size: [0, 1, 2]
Sequential Time: 0.0000 seconds
Pool Time: 0.0286 seconds
Process Time: 0.0433 seconds
Data Size: [0, 1, 2, 3]
Sequential Time: 0.0000 seconds
Pool Time: 0.0262 seconds
Process Time: 0.0436 seconds
Data Size: [0, 1, 2, 3, 4]
Sequential Time: 0.0000 seconds
Pool Time: 0.0290 seconds
Process Time: 0.0455 seconds
Data Size: [0, 1, 2, 3, 4, 5]
Sequential Time: 0.0000 seconds
Pool Time: 0.0272 seconds
Process Time: 0.0450 seconds


## 7. Add matrix




In [11]:
import time
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool, Process
import threading

def sequential_matrix(A,B):
  result = np.zeros_like(A)
  for i in range(len(A)):
    for j in range(len(A[0])):
      result[i][j] = A[i][j] + B[i][j]
  return result

def pool_matrix(A,B):
  with Pool() as pool:
    result = pool.starmap(np.add, zip(A,B))
  return result

def process_matrix(A,B):
  def worker(a, b, result, idx):
    result[idx] = np.add(a,b)

  result = mp.Manager().list([None]*len(A))
  processes = [Process(target = worker, args = (A[i],B[i],result, i)) for i in range(len(A))]

  for p in processes:
    p.start()

  for p in processes:
    p.join()

  return np.array(result)

def threaded_matrix(A,B):
  def worker(a,b, result, idx):
    result[idx] = np.add(a,b)

  result = [None]*len(A)
  threads = [threading.Thread(target = worker, args = (A[i],B[i],result, i)) for i in range(len(A))]

  for t in threads:
    t.start()

  for t in threads:
    t.join()

  return np.array(result)

if __name__ == "__main__":
  A = np.random.randint(0,100, (1000, 1000))
  B = np.random.randint(0, 100, (1000, 1000))

  start = time.time()
  sequential_matrix(A,B)
  seq_time = time.time()-start

  start = time.time()
  pool_matrix(A,B)
  pool_time = time.time()-start

  start = time.time()
  process_matrix(A,B)
  process_time = time.time()-start

  start = time.time()
  threaded_matrix(A,B)
  threaded_time = time.time()-start

  print(f"Sequential Time: {seq_time:.4f} seconds")
  print(f"Pool Time: {pool_time:.4f} seconds")
  print(f"Process Time: {process_time:.4f} seconds")
  print(f"Thread Time: {threaded_time:.4f} seconds")


Sequential Time: 0.7110 seconds
Pool Time: 0.1336 seconds
Process Time: 14.1666 seconds
Thread Time: 0.0958 seconds


## 8. Multiply Matrix

In [15]:
import time
import multiprocessing as mp
import threading
import numpy as np


def multiply_row(row,B):
  return [sum(row[k]*B[k][j] for k in range(len(B))) for j in range(len(B[0]))]

def sequential_matrix(A,B):
  result = np.zeros_like(A)
  for i in range(len(A)):
    for j in range(len(B[0])):
      for k in range(len(B)):
        result[i][j] += A[i][k] * B[k][j]
  return result

def pool_matrix(A,B):
  with mp.Pool() as pool:
    result = pool.starmap(multiply_row, [(A[i], B) for i in range(len(A))])
  return np.array(result)

def process_matrix(A,B):
  def worker(a, b, result, idx):
    result[idx] = [sum(a[k]*b[k][j] for k in range(len(B))) for j in range(len(B[0]))]

  result =  mp.Manager().list([None]*len(A))
  processes = [mp.Process(target = worker, args = (A[i],B,result, i)) for i in range(len(A))]

  for p in processes:
    p.start()
  for p in processes:
    p.join()

  return np.array(result)


def threaded_matrix(A,B):
  def worker(a, b, result, idx):
    result[idx] = [sum(a[k]*b[k][j] for k in range(len(B))) for j in range(len(B[0]))]

  result = list([None]*len(A))
  processes = [threading.Thread(target = worker, args = (A[i],B,result, i)) for i in range(len(A))]

  for t in processes:
    t.start()
  for t in processes:
    t.join()

  return np.array(result)

if __name__ == "__main__":
  A = np.random.randint(0,100, (100, 100))
  B = np.random.randint(0, 100, (100, 100))

  start = time.time()
  sequential_matrix(A,B)
  seq_time = time.time()-start

  start = time.time()
  pool_matrix(A,B)
  pool_time = time.time()-start

  start = time.time()
  process_matrix(A,B)
  process_time = time.time()-start

  start = time.time()
  threaded_matrix(A,B)
  threaded_time = time.time()-start

  print(f"Sequential Time: {seq_time:.4f} seconds")
  print(f"Pool Time: {pool_time:.4f} seconds")
  print(f"Process Time: {process_time:.4f} seconds")
  print(f"Thread Time: {threaded_time:.4f} seconds")



Sequential Time: 0.8206 seconds
Pool Time: 0.4955 seconds
Process Time: 1.8915 seconds
Thread Time: 0.4275 seconds


## 9. Inverse Matrix

In [18]:
import time
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import matplotlib.pyplot as plt

def inverse_matrix(matrix):
    return np.linalg.inv(matrix)

def inverse_matrix_sequential(matrix):
    result = inverse_matrix(matrix)
    return result

def inverse_matrix_process(matrix):
    def worker(matrix, result_list):
        result_list.append(inverse_matrix(matrix))

    with mp.Manager() as manager:
        result_list = manager.list()
        p = mp.Process(target=worker, args=(matrix, result_list))
        p.start()
        p.join()
        result = result_list[0]

    return result

def inverse_matrix_pool(matrix):
    with Pool(processes=mp.cpu_count()) as pool:
        result = pool.apply(inverse_matrix, (matrix,))
    return result

def measure_time(func, matrix):
    start_time = time.time()
    result = func(matrix)
    end_time = time.time()
    return end_time - start_time, result

# def plot_execution_time(data_sizes, sequential_times, process_times, pool_times):
#     plt.figure(figsize=(10, 6))
#     plt.plot(data_sizes, sequential_times, label="Sequential", marker="o")
#     plt.plot(data_sizes, process_times, label="Multiprocessing (Process)", marker="o")
#     plt.plot(data_sizes, pool_times, label="Multiprocessing (Pool)", marker="o")

#     plt.xlabel('Matrix Size (n x n)')
#     plt.ylabel('Execution Time (seconds)')
#     plt.title('Execution Time vs Matrix Size for Inverse Operation')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     process_speedup = [sequential_times[i] / process_times[i] for i in range(len(data_sizes))]
#     pool_speedup = [sequential_times[i] / pool_times[i] for i in range(len(data_sizes))]

#     plt.figure(figsize=(10, 6))
#     plt.plot(data_sizes, [1]*len(data_sizes), label="Sequential (Speed-up = 1)", marker="o", linestyle='--')
#     plt.plot(data_sizes, process_speedup, label="Speed-up (Process)", marker="o")
#     plt.plot(data_sizes, pool_speedup, label="Speed-up (Pool)", marker="o")

#     plt.xlabel('Matrix Size (n x n)')
#     plt.ylabel('Speed-up (Sequential Time / Method Time)')
#     plt.title('Speed-up vs Matrix Size for Inverse Operation')
#     plt.legend()
#     plt.grid(True)
#     plt.show()

if __name__ == "__main__":
    data_sizes = [5000]  # Size of the matrix
    sequential_times = []
    process_times = []
    pool_times = []

    for size in data_sizes:
        matrix = np.random.rand(size, size) * 10

        seq_time, _ = measure_time(inverse_matrix_sequential, matrix)
        sequential_times.append(seq_time)

        proc_time, _ = measure_time(inverse_matrix_process, matrix)
        process_times.append(proc_time)

        pool_time, _ = measure_time(inverse_matrix_pool, matrix)
        pool_times.append(pool_time)

        print(f"Matrix size: {size}x{size}, Sequential Time: {seq_time:.4f}, "
              f"Process Time: {proc_time:.4f}, Pool Time: {pool_time:.4f}")

    # plot_execution_time(data_sizes, sequential_times, process_times, pool_times)

Matrix size: 5000x5000, Sequential Time: 9.8135, Process Time: 10.6668, Pool Time: 11.3954


## Transpose of matrix

In [19]:
import time
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import matplotlib.pyplot as plt
import numpy as np

def transpose_chunk(matrix, start, end):
    result = []
    for i in range(start, end):
        result.append([matrix[j][i] for j in range(len(matrix))])
    return result

def transpose_sequential(matrix):
    result = []
    for i in range(len(matrix[0])):
        time.sleep(0.01)
        result.append([matrix[j][i] for j in range(len(matrix))])
    return result

def transpose_threading(matrix):
    with ThreadPoolExecutor() as executor:
        return list(executor.map(lambda i: transpose_chunk(matrix, i, i+1), range(len(matrix[0]))))

def process_chunk(chunk):
    matrix, start, end = chunk
    return transpose_chunk(matrix, start, end)

def transpose_multiprocessing_process_chunk(matrix):
    n = len(matrix[0])
    chunk_size = n // multiprocessing.cpu_count()

    chunks = [(matrix, i, min(i + chunk_size, n)) for i in range(0, n, chunk_size)]

    with ProcessPoolExecutor() as executor:
        results = executor.map(process_chunk, chunks)

    transposed_matrix = []
    for result in results:
        transposed_matrix.extend(result)
    return transposed_matrix

def transpose_multiprocessing_pool_chunk(matrix):
    n = len(matrix[0])
    chunk_size = n // multiprocessing.cpu_count()

    chunks = [(matrix, i, min(i + chunk_size, n)) for i in range(0, n, chunk_size)]

    with multiprocessing.Pool() as pool:
        results = pool.map(process_chunk, chunks)

    transposed_matrix = []
    for result in results:
        transposed_matrix.extend(result)
    return transposed_matrix

def measure_execution_time(func, matrix):
    start_time = time.time()
    result = func(matrix)
    end_time = time.time()
    return end_time - start_time, result

rows, cols = 2000, 2000
matrix = np.random.randint(1, 100, size=(rows, cols)).tolist()

seq_time, _ = measure_execution_time(transpose_sequential, matrix)
thread_time, _ = measure_execution_time(transpose_threading, matrix)
proc_time, _ = measure_execution_time(transpose_multiprocessing_process_chunk, matrix)
pool_time, _ = measure_execution_time(transpose_multiprocessing_pool_chunk, matrix)

print(f"Sequential execution time: {seq_time:.4f} seconds")
print(f"Multithreading execution time: {thread_time:.4f} seconds")
print(f"Multiprocessing (Process) execution time: {proc_time:.4f} seconds")
print(f"Multiprocessing (Pool) execution time: {pool_time:.4f} seconds")

# speedups = [seq_time / seq_time, seq_time / thread_time, seq_time / proc_time, seq_time / pool_time]

# labels = ['Sequential', 'Multithreading', 'Multiprocessing (Process)', 'Multiprocessing (Pool)']
# times = [seq_time, thread_time, proc_time, pool_time]

# def plot_execution_times(times, labels):
#     plt.figure(figsize=(10, 6))

#     plt.bar(labels, times, color = ['red', 'green', 'blue', 'orange'])

#     plt.title('Execution Time Comparison')
#     plt.xlabel('Method')
#     plt.ylabel('Time (seconds)')
#     plt.grid(True)
#     plt.show()

# def plot_speedup(speedups, labels):
#     plt.figure(figsize=(10, 6))
#     plt.bar(labels, speedups, color=['red', 'green', 'blue', 'orange'])
#     plt.title('Speed-up Comparison')
#     plt.xlabel('Method')
#     plt.ylabel('Speed-up (Sequential Time / Parallel Time)')
    # plt.grid(True)
    # plt.show()

# plot_execution_times(times, labels)
# plot_speedup(speedups, labels)

Sequential execution time: 20.8605 seconds
Multithreading execution time: 0.8296 seconds
Multiprocessing (Process) execution time: 0.8873 seconds
Multiprocessing (Pool) execution time: 0.9574 seconds


## Palindrome

In [ ]:
import time
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import matplotlib.pyplot as plt

def is_palindrome(num, add_delay=False):
    if add_delay:
        time.sleep(0.1)
    return str(num) == str(num)[::-1]

def find_palindromes_sequential(numbers):
    return [num for num in numbers if is_palindrome(num, add_delay=True)]  # Delay added

def find_palindromes_threading(numbers):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(is_palindrome, numbers))
    return [num for num, is_pal in zip(numbers, results) if is_pal]

def find_palindromes_multiprocessing_process(numbers):
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(is_palindrome, numbers))
    return [num for num, is_pal in zip(numbers, results) if is_pal]

def find_palindromes_multiprocessing_pool(numbers):
    with multiprocessing.Pool() as pool:
        results = pool.map(is_palindrome, numbers)
    return [num for num, is_pal in zip(numbers, results) if is_pal]

def measure_execution_time(func, numbers):
    start_time = time.time()
    result = func(numbers)
    end_time = time.time()
    return end_time - start_time, result

numbers = list(range(100000, 100050))

seq_time, _ = measure_execution_time(find_palindromes_sequential, numbers)
thread_time, _ = measure_execution_time(find_palindromes_threading, numbers)
proc_time, _ = measure_execution_time(find_palindromes_multiprocessing_process, numbers)
pool_time, _ = measure_execution_time(find_palindromes_multiprocessing_pool, numbers)

print(f"Sequential execution time (with delay): {seq_time:.4f} seconds")
print(f"Multithreading execution time: {thread_time:.4f} seconds")
print(f"Multiprocessing (Process) execution time: {proc_time:.4f} seconds")
print(f"Multiprocessing (Pool) execution time: {pool_time:.4f} seconds")

# speedups = [seq_time / seq_time, seq_time / thread_time, seq_time / proc_time, seq_time / pool_time]

# labels = ['Sequential', 'Multithreading', 'Multiprocessing (Process)', 'Multiprocessing (Pool)']
# times = [seq_time, thread_time, proc_time, pool_time]

# def plot_execution_times(times, labels):
#     plt.figure(figsize=(10, 6))
#     plt.plot(labels, times, marker='o', color='b')
#     plt.title('Execution Time Comparison')
#     plt.xlabel('Method')
#     plt.ylabel('Time (seconds)')
#     plt.grid(True)
#     plt.show()

# def plot_speedup(speedups, labels):
    # plt.figure(figsize=(10, 6))
    # plt.bar(labels, speedups, color=['red', 'green', 'blue', 'orange'])
    # plt.title('Speed-up Comparison')
    # plt.xlabel('Method')
    # plt.ylabel('Speed-up (Sequential Time / Parallel Time)')
    # plt.grid(True)
    # plt.show()

# plot_execution_times(times, labels)
# plot_speedup(speedups, labels)